# Alignment and Preprocessing
Once the data is made available via `intake` as detailed in the  [Data_Ingestion_with_Intake](./02_Data_Ingestion_with_Intake.ipynb) user guide, the next step is to ensure the data has been appropriately reshaped and aligned across data sources for consumption by the machine learning pipeline, which you can learn about in the next user guide [Machine_Learning](./04_Machine_Learning.ipynb).

In [ ]:
import intake
import numpy as np
import xarray as xr

import cartopy.crs as ccrs

import hvplot.xarray
import holoviews as hv
hv.extension('bokeh', width=80)

## Recap: Loading data

In [ ]:
cat = intake.open_catalog('../catalog.yml')
l5_da = cat.l5().read_chunked()
l5_da

In [ ]:
l8_da = cat.l8().read_chunked()
l8_da

We can use this EPSG value shown above under the ``crs`` key to create a cartopy coordinate reference system that we will be using later on in this notebook:

In [ ]:
crs=ccrs.epsg(32611)

## Preprocessing
The first step in processing data is to remove the missing values. In this case the xarray self-reports the values assigned to `nodatavals`. We can use this information to set the missing values to `NaN`.

In [ ]:
l5_da = l5_da.where(l5_da > l5_da.nodatavals[0])
l8_da = l8_da.where(l8_da > l8_da.nodatavals[0])

We can make sure that no more -9999s show up in the data, by calculating the minimum value in each dataarray as follows:

In [ ]:
l5_da.min().compute()

In [ ]:
l8_da.min().compute()

**NOTE:** These operations take a non-trivial amount of time because they require that the data actually be loaded. 

## Compute NDVI

Now we will calculate NDVI for each of these image sets and persist the output in memory for speedy calculations later.

In [ ]:
NDVI_1988 = (l5_da.sel(band=5) - l5_da.sel(band=4)) / (l5_da.sel(band=5) + l5_da.sel(band=4)).persist()
NDVI_1988

In [ ]:
NDVI_2017 = (l8_da.sel(band=5) - l8_da.sel(band=4)) / (l8_da.sel(band=5) + l8_da.sel(band=4)).persist()
NDVI_2017

**NOTE:** Since we did this operations on just one band we lost some of the helpful metadata on our `xarray` objects. We can get that metadata back by copying the structure of the one of the original array bands and creating a new array with that structure that the NDVI data.

In [ ]:
NDVI_1988 = l5_da.sel(band=4).drop('band').copy(data=NDVI_1988)
NDVI_2017 = l8_da.sel(band=4).drop('band').copy(data=NDVI_2017)
NDVI_2017

## Aligning the data

These two sets of landsat bands cover roughly the same area but were taken in 1988 and 2017. While they have the same resolution (30m) they have different numbers of grid cells and were taken at slightly different angles (transform).

In [ ]:
NDVI_1988.transform is not NDVI_2017.transform

When these data are combined onto one dataset, the shape of the dataset grows to be the union of the dimensions on each array. 

In [ ]:
ds = xr.Dataset({'1988': NDVI_1988, '2017': NDVI_2017})
ds

We can quickly subset to one point, such as the center of Walker lake: 38.6942° N, 118.7081° W

In [ ]:
x_center, y_center = crs.transform_point(-118.7081, 38.6942, ccrs.PlateCarree())

In [ ]:
ds.sel(x=x_center, y=y_center, method='nearest')

Or we can subset the area around this central point as follows:

In [ ]:
buffer = 1.5e4

In [ ]:
subset = ds.sel(x=slice(x_center-buffer, x_center+buffer), y=slice(y_center-buffer, y_center+buffer))
subset

When these data are loaded into geoviews, the transform is automatically applied and the data can be overlayed on top of each other or operations can be performed on matching grid cells.

In [ ]:
%%opts Image [width=500 height=500] (cmap='viridis')

NDVI_1988_p = subset['1988'].hvplot(crs=crs, clim=(-3, 1)).relabel('1988')
NDVI_2017_p = subset['2017'].hvplot(crs=crs, clim=(-3, 1)).relabel('2017')

hv.Layout(NDVI_1988_p + NDVI_2017_p).options(shared_axes=True)

In [ ]:
%%opts Image [width=600 height=500] (cmap='coolwarm')

(subset['1988'] - subset['2017']).hvplot(crs=crs, clim=(-2, 2)).relabel('Difference in NDVI')

In [ ]:
from datashader import resampl

See [Walker_Lake](../Walker_Lake.ipynb) for more work on calculating the difference between the water levels over time.